In [2]:
# -------------------------------
# Forward Chaining in First Order Logic
# Example: Prove that Robert is a Criminal
# -------------------------------

from itertools import product

# -------------------------------
# Knowledge Base (from your slides)
# -------------------------------

facts = set([
    "American(Robert)",
    "Missile(T1)",
    "Owns(A,T1)",
    "Enemy(A,America)"
])

rules = [
    (["American(p)", "Weapon(q)", "Sells(p,q,r)", "Hostile(r)"], "Criminal(p)"),
    (["Missile(x)"], "Weapon(x)"),
    (["Owns(A,x)", "Missile(x)"], "Sells(Robert,x,A)"),
    (["Enemy(x,America)"], "Hostile(x)")
]

goal = "Criminal(Robert)"


# -------------------------------
# Helper Functions
# -------------------------------

def parse(expr):
    """Split predicate and arguments."""
    name, args = expr.split("(")
    args = args[:-1].split(",")
    return name, args


def unify(expr1, expr2, theta=None):
    """Unify two expressions and return substitution map or None."""
    if theta is None:
        theta = {}

    name1, args1 = parse(expr1)
    name2, args2 = parse(expr2)

    if name1 != name2 or len(args1) != len(args2):
        return None

    for a1, a2 in zip(args1, args2):
        if a1 == a2:
            continue
        elif a1.islower():  # a1 is variable
            if a1 in theta and theta[a1] != a2:
                return None
            theta[a1] = a2
        elif a2.islower():  # a2 is variable
            if a2 in theta and theta[a2] != a1:
                return None
            theta[a2] = a1
        else:
            return None
    return theta


def substitute(expr, subs):
    """Apply substitutions to expression."""
    name, args = parse(expr)
    new_args = [subs.get(a, a) for a in args]
    return f"{name}({','.join(new_args)})"


# -------------------------------
# Forward Chaining Algorithm
# -------------------------------

def forward_chain(facts, rules, goal):
    new_facts = set(facts)

    while True:
        added = set()

        for premises, conclusion in rules:
            # find all combinations of facts that could match premises
            fact_groups = [list(facts) for _ in premises]
            for combo in product(*fact_groups):
                subs = {}
                valid = True
                for p, f in zip(premises, combo):
                    theta = unify(p, f, subs.copy())
                    if theta is None:
                        valid = False
                        break
                    subs.update(theta)
                if valid:
                    new_fact = substitute(conclusion, subs)
                    if new_fact not in new_facts:
                        print(f"Derived new fact: {new_fact}")
                        added.add(new_fact)
                        if new_fact == goal:
                            print("\n✅ Goal Reached!")
                            return True

        if not added:
            break
        new_facts.update(added)
        facts = new_facts

    return False


# -------------------------------
# Run Forward Chaining
# -------------------------------

print("Initial Facts:")
for f in facts:
    print("  ", f)

print("\nApplying Forward Chaining...\n")

if forward_chain(facts, rules, goal):
    print("\nConclusion: Robert is a Criminal ✅")
else:
    print("\nConclusion: Could not prove that Robert is a Criminal ❌")


Initial Facts:
   Owns(A,T1)
   Missile(T1)
   American(Robert)
   Enemy(A,America)

Applying Forward Chaining...

Derived new fact: Weapon(T1)
Derived new fact: Sells(Robert,T1,A)
Derived new fact: Hostile(A)
Derived new fact: Criminal(Robert)

✅ Goal Reached!

Conclusion: Robert is a Criminal ✅
